In [1]:
import os
%pwd

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project\\research'

In [4]:
os.chdir("c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project")

In [5]:
%pwd

'c:\\Code Playground\\Programs_github\\Chicken-Disease-Classification-project'

In [6]:
# Update the Entity Linking model with the new data

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig: 
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig: 
        config  = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )

        return prepare_base_model_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
class PrepareBaseModel: 
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self): 
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size, 
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def  _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate): 
        if freeze_all: 
            for layer in model.layers: 
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0): 
            for layer in model.layers[:-freeze_till]: 
                model.trainable = False

        flatten_layer = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation='softmax')(flatten_layer)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(), 
            metrics = ['accuracy']
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self): 
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True, 
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
# Create the pipeline for the base model
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-07-29 23:00:54,543: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-07-29 23:00:54,556: INFO: common] yaml file: params.yaml loaded successfully
[2023-07-29 23:00:54,558: INFO: common] directory created at: artifacts
[2023-07-29 23:00:54,560: INFO: common] directory created at: artifacts/prepare_base_model
58889256/58889256 [==============================] - 11s 0us/step
[2023-07-29 23:01:10,744: WARNING: saving_utils] Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                     

c:\Users\rahul\miniconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
